In [1]:
#=============================================#
#                 FUNCTIONS                   #
#=============================================#

def coeffProbs(prob_mutation):
    prob_ab = (1 - prob_mutation)/2
    probs = [prob_ab, prob_ab, prob_mutation]
    prob_dist = GeneralDiscreteDistribution(probs)
    return prob_dist.get_random_element()


def genSplittingField(f,n):
    linear_factors = factor(nest(f,n,x) - x)
    K.<a> = NumberField(linear_factors[len(linear_factors)-1][0])
    return K

def display_progress(item, length, message):
    from IPython.display import display, clear_output
    clear_output(wait=True)
    display(message + "... " + str(item) +"/" + str(length))

def initialGen(K, P, D, popsize, display = False):
    generation = []
    while len(generation) < popsize:
        alpha = K.random_element(den_bound = 1)
        if not P(alpha).is_preperiodic(D):
            generation.append([alpha, D.canonical_height(P(alpha))])
            if display:
                display_progress(len(generation), popsize, "generating initial population")
    generation.sort(key = lambda x: x[1])
    return generation

def genCombs(n,k):
    C = Combinations(list(range(n)),k)
    combList = []
    for i in range(C.cardinality()):
        combList.append(C.list()[i])
    return combList

def breedElements(alpha,beta, K, mutation_chance):
    gamma = 0
    deg = max(alpha.polynomial().degree(), beta.polynomial().degree())
    for i in range(deg + 1):
        parentCoeff = coeffProbs(mutation_chance)
        if parentCoeff == 2:
            newCoeff = int(RealDistribution('uniform', [-10,10]).get_random_element())
        elif parentCoeff == 1:
            newCoeff = alpha[i]
        else:
            newCoeff = beta[i]
        gamma += K.an_element()^i*newCoeff
    return gamma

def breedGen(parentGen, genKeep, genCombs, K, P, D, mutation_chance):
    childGen = []
    for i in range(genKeep):
        childGen.append(parentGen[i])
    for i in range(len(parentGen) - genKeep):
        alpha = parentGen[genCombs[i][0]][0]
        beta = parentGen[genCombs[i][1]][0]
        gamma = breedElements(alpha,beta,K,mutation_chance)
        childGen.append([gamma, D.canonical_height(P(gamma))])
    childGen.sort(key = lambda x: x[1])
    return childGen

#=============[MAIN]=============#

#def main():
#    R.<x> = PolynomialRing(ZZ)
#    K = genSplittingField(x^2-1,3)
#    P.<X,Y> = ProjectiveSpace(K,1)
#    D = DynamicalSystem([X^2 - Y^2, Y^2])
#    initial = initialGen(K, P, D, 10, True)

#if __name__ == "__main__":
#    main()
    

            